## Scraping the Internet Archive

Mich interessiert, welche Artikel wann auf der Startseite der NZZ ausgespielt wurden, und zwar retrospektiv für das Jahr 2020 oder sogar noch länger zurück. 

Snapshots (auch 'Captures') von www.nzz.ch sind im Internet Archive hinterlegt. Ich baue einen Scraper, der diese Snapshots abspeichert und eine Tabelle erstellt mit Titeln, Teasern und URL. Anschliessend durchsuche ich diese Datenbank nach bestimmten Schlagworten.

Zum Beispiel können wir mit diesem Datensatz
* die mediale Karriere von Trump nachzeichnen: Wann beherrschte er die News? Und weshalb?
* das Coronajahr analysieren: Wann standen Tests, Impfungen, Lockdown, Kinder & Homeschoooling, Homeoffice, Psyche und Krise im Vordergrund?

Das Scraping findet in drei Stufen statt:
1. Wir erstellen eine Liste der URLs der einzelnen Snapshots eines Jahres. 
2. Wir rufen die URLs auf der Liste mit dem Scraper auf und speichern das html ab (eines pro Capture)
3. Wir erstellen aus dem html eine Liste mit Titeln, Teasern und Artikel-URL und speichern diese ebenfalls ab (auch hier eines pro Capture)

Die Analyse findet im Notebook CoronaTrendsSchweiz.ipynb statt:
1. Suche in den Listen aus Scraping-3 nach passenden Schlagworten und schreibe diese in eine Liste (Datum - Trefferzahl)
2. Visualisiere als Steamplot
3. Füge bei Ereignissen eine Annotation an

In [7]:
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from random import randint
from warnings import warn
import json
import pandas as pd

**1. Liste aller Captures der NZZ-Frontpage im Internet Archive**

In [8]:
# Liste der URL aller NZZ captures in der Wayback machine.
# Dafür gibt es eine API: https://github.com/internetarchive/wayback/tree/master/wayback-cdx-server

# Suche via API-Abfrage auf 2020 eingrenzen
url = 'http://web.archive.org/cdx/search/cdx?url=nzz.ch&from=20200101&to=20201231&output=json'   
urls = rq.get(url).text

# Aus dem json müssen wir die Timestamps extrahieren und diese dann wieder zur URL des Captures zusammenfügen:
parse_url = json.loads(urls) 

url_list = []
for i in range(1,len(parse_url)):
    tstamp = parse_url[i][1]
    waybacklink = 'https://web.archive.org/web/'+ tstamp +'/https://www.nzz.ch/'
    url_list.append(waybacklink)

# alle erzeugten URL in ein txt-file schreiben:    
with open('IA_captures_nzz_fp.txt', 'w') as filehandle:
    json.dump(url_list, filehandle)

In [9]:
len(url_list)

7078

**2. Code zum Scrapen der Captures**

In [23]:
# HTML aufrufen, parsen, abspeichern 

for url in url_list[320:]:
    page = rq.get(url).text
    soup = bs(page, 'html.parser')
    html = soup.prettify("utf-8")

    timestamp = url.split('/')[4]
    with open(f"htmls/NZZ-frontpage-{timestamp}.html", "wb") as file:
        file.write(html)

# Titel, Teaser und url aller Artikel herausziehen
    
    articles = soup.find_all('div', {'class' : "teaser__content"})
    data = []
    for article in articles:
        url = article.find('a')["href"].split('//')[1]
        try:
            title = article.find('h2', {'class' : "teaser__title"}).text.replace("\n", "")
        except:
            title = 'n/a'
        try:
            teaser = article.find('div', {'class' : "teaser__lead"}).text.replace("\n", "")
        except:
            teaser = 'n/a'
        
        mini_dict = {'Titel': title,
                     'Teaser': teaser,
                     'url': url}
        data.append(mini_dict)
        pd.DataFrame(data).to_csv(f'csvs/{timestamp}.csv')